# Flipkart Laptop Sales - Analysis

Data source: https://www.kaggle.com/datasets/mrmars1010/flipkart-product-datalaptops

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flipkart-product-datalaptops/Flipkart-Laptops.xlsx


In [25]:
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [3]:
df_raw = pd.read_excel("/kaggle/input/flipkart-product-datalaptops/Flipkart-Laptops.xlsx")

In [4]:
df_raw.head()

,Product Name,ProductID,Product image,Actual price,Discount price,Stars,Rating,Reviews,Description,Link
0,MSI Cyborg 15 Intel Core i5 12th Gen 12450H - ...,COMGZW35W3DSJADN,NaN,89990,54990,3.9,7 Ratings,1 Reviews,Intel Core i5 Processor (12th Gen)16 GB DDR5 R...,https://www.flipkart.com/msi-cyborg-15-intel-c...
1,MSI Thin 15 Intel Core i7 12th Gen 12650H - (8...,COMGZW37ZX66DBHF,NaN,83990,67990,NIL,NIL,NIL,Intel Core i7 Processor (12th Gen)8 GB DDR4 RA...,https://www.flipkart.com/msi-thin-15-intel-cor...
2,DELL Inspiron 3520 Intel Core i3 12th Gen 1215...,COMGJ75HJGFDJ6JN,NaN,49240,35660,4.2,"1,805 Ratings",143 Reviews,Intel Core i3 Processor (12th Gen)8 GB DDR4 RA...,https://www.flipkart.com/dell-inspiron-3520-in...
3,Acer One (2024) Intel Core i3 11th Gen 1115G4 ...,COMGPF5CQ7VDWDT4,NaN,43999,26990,4.2,"6,977 Ratings",596 Reviews,Intel Core i3 Processor (11th Gen)8 GB DDR4 RA...,https://www.flipkart.com/acer-one-2024-intel-c...
4,Lenovo V15 AMD Ryzen 3 Quad Core 7320U - (8 GB...,COMGPYKZAWY8UX6C,NaN,59400,27989,4.2,"1,263 Ratings",113 Reviews,AMD Ryzen 3 Quad Core Processor8 GB LPDDR5 RAM...,https://www.flipkart.com/lenovo-v15-amd-ryzen-...


In [5]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Product Name    960 non-null    object 
 1   ProductID       960 non-null    object 
 2   Product image   0 non-null      float64
 3   Actual price    960 non-null    object 
 4   Discount price  960 non-null    object 
 5   Stars           960 non-null    object 
 6   Rating          960 non-null    object 
 7   Reviews         960 non-null    object 
 8   Description     960 non-null    object 
 9   Link            960 non-null    object 
dtypes: float64(1), object(9)
memory usage: 75.1+ KB


In [6]:
for c in ['Actual price', 'Discount price', 'Stars']:
    us = df_raw[df_raw[c].apply(lambda a: not str(a).isnumeric())][c].unique()
    print(f"{c} -> {us}")

Actual price -> ['NIL']
Discount price -> [' ']
Stars -> [3.9 'NIL' 4.2 4.6 2.3 4.3 4.5 3.8 4.1 4.7 4.4 3.7 4.8 3.2 3.6 3.4 3.5 3.3
 2.8 4.9 3.1]


In [7]:
us = df_raw[df_raw['Rating'].apply(lambda a: not a.__contains__("Ratings"))]["Rating"].unique()
print(f"Rating -> {us}")

us = df_raw[df_raw['Reviews'].apply(lambda a: not a.__contains__("Reviews"))]["Reviews"].unique()
print(f"Reviews -> {us}")

Rating -> ['NIL']
Reviews -> ['NIL']


In [8]:
df_raw[(df_raw['Discount price'] == ' ') | (df_raw['Actual price'] == 'NIL')]

,Product Name,ProductID,Product image,Actual price,Discount price,Stars,Rating,Reviews,Description,Link
149,HP Victus Intel Core i7 12th Gen 12650H - (16 ...,COMH2DYZGEUJ4T5S,NaN,NIL,90898,NIL,NIL,NIL,Intel Core i7 Processor (12th Gen)16 GB DDR4 R...,https://www.flipkart.com/hp-victus-intel-core-...
473,HP AMD Ryzen 3 Quad Core 5300u - (8 GB/512 GB ...,COMGT84XFZSQHVN8,NaN,NIL,39631,NIL,NIL,NIL,AMD Ryzen 3 Quad Core Processor8 GB DDR4 RAM64...,https://www.flipkart.com/hp-amd-ryzen-3-quad-c...
644,Lenovo IdeaPad 3 Intel Core i3 12th Gen 1215U ...,COMGWVBHBWRRFTZB,NaN,NIL,,4.3,4 Ratings,0 Reviews,Intel Core i3 Processor (12th Gen)8 GB DDR4 RA...,https://www.flipkart.com/lenovo-ideapad-3-inte...
820,HP Victus AMD Ryzen 5 Hexa Core 5600H - (8 GB/...,COMGGYEBBQFFPTTN,NaN,NIL,,4.2,70 Ratings,7 Reviews,AMD Ryzen 5 Hexa Core Processor8 GB DDR4 RAM64...,https://www.flipkart.com/hp-victus-amd-ryzen-5...
860,HP Intel Core N N305 - (8 GB/256 GB/Chrome Plu...,COMH2V5YJGNRZFFQ,NaN,NIL,56963,NIL,NIL,NIL,Intel Core N Processor8 GB LPDDR5 RAMChrome Op...,https://www.flipkart.com/hp-intel-core-n-n305-...
874,HP Victus Intel Core i5 12th Gen 12450H - (16 ...,COMH2DYZUABHHQZA,NaN,NIL,77566,NIL,NIL,NIL,Intel Core i5 Processor (12th Gen)16 GB DDR4 R...,https://www.flipkart.com/hp-victus-intel-core-...
956,HP Victus Intel Core i5 12th Gen 12450H - (16 ...,COMH2DYZHMHZ5UPG,NaN,NIL,82414,NIL,NIL,NIL,Intel Core i5 Processor (12th Gen)16 GB DDR4 R...,https://www.flipkart.com/hp-victus-intel-core-...


In [9]:
df_raw[(df_raw['Rating'] == 'NIL') | (df_raw['Reviews'] == 'NIL') | (df_raw['Stars'] == 'NIL')]

,Product Name,ProductID,Product image,Actual price,Discount price,Stars,Rating,Reviews,Description,Link
1,MSI Thin 15 Intel Core i7 12th Gen 12650H - (8...,COMGZW37ZX66DBHF,NaN,83990,67990,NIL,NIL,NIL,Intel Core i7 Processor (12th Gen)8 GB DDR4 RA...,https://www.flipkart.com/msi-thin-15-intel-cor...
10,ASUS TUF Gaming A15 (2024) AMD Ryzen 7 Octa Co...,COMGZW3FGTJMSNNM,NaN,143990,124990,NIL,NIL,NIL,AMD Ryzen 7 Octa Core Processor16 GB DDR5 RAMW...,https://www.flipkart.com/asus-tuf-gaming-a15-2...
11,ASUS TUF Gaming A15 (2024) AMD Ryzen 9 Octa Co...,COMGZW3F57BZCGQC,NaN,171990,142990,NIL,NIL,NIL,AMD Ryzen 9 Octa Core Processor16 GB DDR5 RAMW...,https://www.flipkart.com/asus-tuf-gaming-a15-2...
16,MSI Sword 16 HX Intel Core i7 14th Gen 14700HX...,COMGYSGNZVWFAHGH,NaN,139990,124990,NIL,NIL,NIL,Intel Core i7 Processor (14th Gen)16 GB DDR5 R...,https://www.flipkart.com/msi-sword-16-hx-intel...
17,MSI Prestige 16 AI Studio Intel Core Ultra 9 1...,COMGXMTAHXSHTQGD,NaN,212990,189990,NIL,NIL,NIL,Intel Core Ultra 9 Processor32 GB LPDDR5 RAMWi...,https://www.flipkart.com/msi-prestige-16-ai-st...
...,...,...,...,...,...,...,...,...,...,...
932,MSI Thin 15 Intel Core i7 12th Gen 12650H - (8...,COMGZW37ZX66DBHF,NaN,83990,67990,NIL,NIL,NIL,Intel Core i7 Processor (12th Gen)8 GB DDR4 RA...,https://www.flipkart.com/msi-thin-15-intel-cor...
936,MSI Commercial 14 H Intel Core i5 13th Gen 134...,COMGYSFG2KNNQY9H,NaN,98990,87990,NIL,NIL,NIL,Intel Core i5 Processor (13th Gen)16 GB DDR4 R...,https://www.flipkart.com/msi-commercial-14-h-i...
941,Acer Nitro AMD Ryzen 7 Octa Core 7735HS - (16 ...,COMGZN7Z8V7AGTAD,NaN,89999,74990,NIL,NIL,NIL,AMD Ryzen 7 Octa Core Processor16 GB DDR5 RAMW...,https://www.flipkart.com/acer-nitro-amd-ryzen-...
953,MSI Thin 15 Intel Core i7 12th Gen 12650H - (8...,COMGZW37ZX66DBHF,NaN,83990,67990,NIL,NIL,NIL,Intel Core i7 Processor (12th Gen)8 GB DDR4 RA...,https://www.flipkart.com/msi-thin-15-intel-cor...


## Notes
- 'Product' image is all null, so it don't have any info
- 'ProductID' have no info we can use
- 'Actual price', 'Discount price', 'Stars', 'Rating', 'Reviews' need to be converted to numeric data. These are only numeric data we can use
- 'Product Name' have brand info and some spec info
- 'Description' is another column from where we can extract specs
- 'Stars' have 'NIL' that should be replaced appropriately. 0 would be a bad replacement as it would mean a bad review. We can replace it with -1, another category implying no reviews yet.
- 'Actual price' and 'Discount price' has non-numeric values like ' ' and 'NIL'. These kind of data is small (7 entries), so can be removed
- 'Rating' and 'Reviews' have non-numeric value like 'NIL'. We can replace them with 0. We should also extract numeric values from them

## Data cleaning

In [10]:
df_cleaned = df_raw.drop(columns=['ProductID', 'Product image'])
df_cleaned.columns = df_cleaned.columns.str.lower().str.replace(" ", "_")

In [11]:
df_cleaned.columns

Index(['product_name', 'actual_price', 'discount_price', 'stars', 'rating',
       'reviews', 'description', 'link'],
      dtype='object')

In [12]:
df_cleaned.drop(df_cleaned[(df_cleaned['discount_price'] == ' ') | (df_cleaned['actual_price'] == 'NIL')].index, inplace=True)
df_cleaned.drop(df_cleaned[(df_cleaned['rating'] == 'NIL') | (df_cleaned['reviews'] == 'NIL') | (df_cleaned['stars'] == 'NIL')].index, inplace=True)

In [13]:
df_cleaned['actual_price'] = df_cleaned['actual_price'].astype(np.float32)
df_cleaned['discount_price'] = df_cleaned['discount_price'].astype(np.float32)

In [14]:
# 4 Ratings
df_cleaned['rating'] = df_cleaned['rating'].apply(lambda s: int(s.split(" ")[0].replace(",", "")))

# 7 Reviews
df_cleaned['reviews'] = df_cleaned['reviews'].apply(lambda s: int(s.split(" ")[0].replace(",", "")))

In [15]:
df_cleaned['stars'] = df_cleaned['stars'].replace("NIL", -1).astype(np.float32)

## Feature extraction

### If we analyze 'product_name' the first word can be found as brand name. We are going to extract that

In [16]:
for i in range(10):
    sdf = df_cleaned["description"].sample(frac=1).reset_index(drop=True)
    print(len(sdf.iloc[i]), sdf.iloc[i])

406 Apple M2 Processor8 GB Unified Memory RAMMac OS Operating System256 GB SSD34.54 cm (13.6 Inch) DisplayBuilt-in Apps: iMovie, Siri, GarageBand, Pages, Numbers, Photos, Keynote, Safari, Mail, FaceTime, Messages, Maps, Stocks, Home, Voice Memos, Notes, Calendar, Contacts, Reminders, Photo Booth, Preview, Books, App Store, Time Machine, TV, Music, Podcasts, Find My, QuickTime Player1 Year Limited WarraÂ­nty
133 Intel Core i5 Processor (12th Gen)16 GB DDR5 RAMWindows 11 Operating System512 GB SSD39.62 cm (15.6 Inch) Display2 Year Warranty Term
219 Intel Core i5 Processor (13th Gen)16 GB LPDDR5 RAMWindows 11 Operating System512 GB SSD35.56 cm (14 Inch) DisplayMicrosoft Office Home & Student 20211Yr Warranty + 1 Yr Premium Care and 1 Yr Accidental Damage Protection
177 Intel Core i5 Processor (11th Gen)8 GB DDR4 RAM64 bit Windows 11 Operating System512 GB SSD40.89 cm (16.1 inch) DisplayMicrosoft Office Home & Student 20191 Year Onsite Warranty
166 Intel Core Ultra 5 Processor16 GB LPDDR5X

In [17]:
df_cleaned['product_name'].apply(lambda pn: pn.split(" ")[0]).unique()

array(['MSI', 'DELL', 'Acer', 'Lenovo', 'HP', 'CHUWI', 'ASUS', 'Thomson',
       'SAMSUNG', 'Ultimus', 'realme', 'ZEBRONICS', 'Infinix', 'Apple',
       'Colorful', 'Primebook', 'Avita', 'AXL', 'GIGABYTE', 'Mi', 'Vaio'],
      dtype=object)

In [18]:
df_cleaned['brand'] = df_cleaned['product_name'].apply(lambda pn: pn.split(" ")[0])

### The laptop processor in this is of following types only. This can be used to extract cpu brand
- Intel
- AMD
- MediaTek
- Qualcomm Snapdragon
- Apple (Apple M2 and Apple M3)

In [19]:
df_cleaned[df_cleaned['description'].apply(lambda d: not d.__contains__("Intel ") and not d.__contains__("AMD ") and not d.__contains__("MediaTek ") 
                                           and not d.__contains__("Qualcomm Snapdragon ") and not d.__contains__("Apple M2 ")and not d.__contains__("Apple M3 "))]['description']

Series([], Name: description, dtype: object)

In [20]:
def get_cpu_brand(d: str):
    if d.__contains__("Intel "):
        return "Intel"
    elif d.__contains__("AMD "):
        return "AMD"
    elif d.__contains__("MediaTek "):
        return "MediaTek"
    elif d.__contains__("Qualcomm Snapdragon "):
        return 'Qualcomm Snapdragon'
    elif d.__contains__("Apple M2 ") or d.__contains__("Apple M3 "):
        return 'Apple Processor'
    else:
        return "Unknown"

In [21]:
df_cleaned["cpu_brand"] = df_cleaned['description'].apply(get_cpu_brand)

In [63]:
df_cleaned['description'].str.extract(r"^(.+) Processor", flags=re.IGNORECASE, expand=True)[0].unique()

array(['Intel Core i5', 'Intel Core i3', 'AMD Ryzen 3 Quad Core',
       'AMD Ryzen 5 Hexa Core', 'AMD Ryzen 7 Octa Core',
       'MediaTek MT8183', 'Intel Core i7', 'Intel Celeron Dual Core', nan,
       'Processor: Intel i3-1215U (Base- 0.9 GHz & Turbo up to 4.40 GHz) 6 CoresRAM & Storage: 8GB DDR4 & 512GB SSDGraphics & Keyboard: Integrated & Standard KeyboardDisplay: 15.6" FHD WVA AG 120Hz 250 nits Narrow BorderIntel Core i3',
       'AMD Athlon Dual Core', 'AMD Ryzen 5 Quad Core',
       'Processor: Intel i5-1235U (Base- 3.30 GHz & Turbo up to 4.40 GHz) 10 CoresRAM & Storage: 8GB DDR4 & 512GB SSDGraphics & Keyboard: Integrated & Standard KeyboardDisplay: 15.6" FHD WVA AG 120Hz 250 nits Narrow BorderIntel Core i5',
       'Intel Celeron Quad Core', 'AMD Ryzen 3 Dual Core',
       'Intel Core Ultra 5', 'Intel Core Ultra 9',
       'AMD Ryzen 3 Octa Core', 'Apple M3',
       'Qualcomm Snapdragon 7c Gen 2', 'Apple M2', 'MediaTek MT8788',
       'Intel Core i9', 'AMD Ryzen 3 Hexa Core',

### RAM

In [56]:
a = df_cleaned['description'].str.extract(r"([0-9]+) GB (.+) RAM", flags=re.IGNORECASE, expand=True)
df_cleaned['ram_size'] = a[0]
df_cleaned["ram_type"] = a[1]

In [54]:
df_cleaned['description'].str.extract(r"([0-9]+) GB (SSD)", flags=re.IGNORECASE, expand=True)

,0,1
count,729,729
unique,3,1
top,512,SSD
freq,637,729


In [43]:
df_cleaned['description'].values

array(['Intel Core i5 Processor (12th Gen)16 GB DDR5 RAMWindows 11 Operating System512 GB SSD39.62 cm (15.6 Inch) Display2 Year Warranty Term',
       'Intel Core i3 Processor (12th Gen)8 GB DDR4 RAM64 bit Windows 11 Operating System512 GB SSD39.62 cm (15.6 Inch) Display1 Year Onsite Hardware Service',
       'Intel Core i3 Processor (11th Gen)8 GB DDR4 RAM64 bit Windows 11 Operating System512 GB SSD35.56 cm (14 Inch) Display1 Year Warranty',
       'AMD Ryzen 3 Quad Core Processor8 GB LPDDR5 RAMWindows 11 Operating System512 GB SSD100.63 cm (39.62 cm) Display1 Year Onsite Warranty',
       'AMD Ryzen 3 Quad Core Processor8 GB DDR4 RAMWindows 11 Operating System512 GB SSD39.62 cm (15.6 Inch) Display1 Year Onsite warranty',
       'Intel Core i3 Processor (12th Gen)8 GB DDR4 RAM64 bit Windows 11 Home Operating System512 GB SSD39.62 cm (15.6 inch) DisplayMicrosoft Office Home & Student 20211 Year Onsite Warranty',
       'AMD Ryzen 5 Hexa Core Processor16 GB LPDDR5 RAMWindows 11 Operatin